In [0]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, BatchNormalization, Activation, add
from keras.models import Model, model_from_json
from keras.optimizers import Adam
from keras.layers.advanced_activations import ELU, LeakyReLU
from keras.utils.vis_utils import plot_model
from keras import backend as K 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
os.chdir("/content/drive/My Drive/CelebAMaskHQ/")
!ls
#img_files = next(os.walk('images/'))[2]
#leye_files = next(os.walk('l_eye/'))[2]
#reye_files = next(os.walk('r_eye/'))[2]
#llip_files = next(os.walk('l_lip/'))[2]
#ulip_files = next(os.walk('u_lip/'))[2]

# Used to display the differences between the number of masks available
# For each folder
#print(len(img_files))
#print(len(leye_files))
#print(len(reye_files))
#print(len(llip_files))
#print(len(ulip_files))

# Checks number of valid images
# CHANGE I FOR EACH ITERATION TYPE
i = 0
count = 0
# Stores image and mask arrays
X2 = []
Y2 = []
pbar = tqdm(total = 10000)
while(i < 10000):    
    img_f1 = "/content/drive/My Drive/CelebAMaskHQ/images/"+str(i)+".jpg"
    if(os.path.exists(img_f1)):
        print("Image "+img_f1+" exists")
        if(os.path.exists('/content/drive/My Drive/CelebAMaskHQ/l_eye/'+str(i)+'_l_eye.png') and os.path.exists('/content/drive/My Drive/CelebAMaskHQ/r_eye/'+str(i)+'_r_eye.png') and os.path.exists('/content/drive/My Drive/CelebAMaskHQ/l_lip/'+str(i)+'_l_lip.png') and os.path.exists('/content/drive/My Drive/CelebAMaskHQ/u_lip/'+str(i)+'_u_lip.png') ):                
            
            # NOTE : The above if case is made in order to compensate for incomplete data
            # i.e there is a lack of masks for certain images, as noted by the 
            # difference of number of file objects per mask in the dataset

            # Taking main image as input and resizing
            img = cv2.imread('images/'+str(i)+'.jpg', cv2.IMREAD_COLOR)
            resized_img = cv2.resize(img,(256, 192), interpolation = cv2.INTER_CUBIC)
        
            X2.append(resized_img)
            # We are reading each and every mask available
            # In accordance to the dataset provided
            leye = cv2.imread('l_eye/{}'.format(str(i)+'_l_eye.png'), cv2.IMREAD_GRAYSCALE)
            reye = cv2.imread('r_eye/{}'.format(str(i)+'_r_eye.png'), cv2.IMREAD_GRAYSCALE)
            llip = cv2.imread('l_lip/{}'.format(str(i)+'_l_lip.png'), cv2.IMREAD_GRAYSCALE)
            ulip = cv2.imread('u_lip/{}'.format(str(i)+'_u_lip.png'), cv2.IMREAD_GRAYSCALE)

            # We resize our data to fit params of MultiResUNet
            resized_leye = cv2.resize(leye,(256, 192), interpolation = cv2.INTER_CUBIC)
            resized_reye = cv2.resize(reye,(256, 192), interpolation = cv2.INTER_CUBIC)
            resized_llip = cv2.resize(llip,(256, 192), interpolation = cv2.INTER_CUBIC)
            resized_ulip = cv2.resize(ulip,(256, 192), interpolation = cv2.INTER_CUBIC)

            # The below statements combine all the four masks into one picture
            # In order to satisfy question requirements(preprocessing)
            op = np.maximum(np.array(resized_leye),np.array(resized_reye))
            op = np.maximum(op, np.array(resized_llip))
            op = np.maximum(op,np.array(resized_ulip))

            count += 1
            Y2.append(op)
            pbar.update(1)
            print("Picture "+str(i)+" is valid")

        i += 1
        print("Iteration "+str(i)+" complete")

np.save('X1',X2)
np.save('Y1',Y2)
print("The number of valid images(images entered) : "+str(count))        
pbar.close()